# Introduction
This is a simple image classifier using the CIFAR-10 dataset.

# Step 1
Installing PyTorch and TorchVision for building the machine learning model and Matplotlib for ploting the results.

In [ ]:
!pip install torch torchvision matplotlib

# Step 2
Importing the necessary libraries

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim

# Check if CUDA is available
if torch.cuda.is_available():
    print(f"CUDA is available! Device: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Check runtime settings.")

# Step 3
Loading the CIFAR-10 dataset

In [ ]:
# Define transformations for the training and test sets
transform = transforms.Compose(
    [transforms.ToTensor(), # converts a PIL image to a tensor and transforms the values {0,...255} --> [0, 1]
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # first tuple -> means, second tuple -> std's. output=(input-mean)/std
# The image has three color channels (RGB) and therefore the tuples have three elements, one for each color


# Download and load the training data
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# Download and load the test data
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Define class names
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Assuming 'net' is your model and 'inputs', 'labels' are your data
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Step 4
Exploring the Data - visualize a few images from the dataset to understand what you are working with:

In [ ]:
# Functions to show an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize - [-1,1] --> [0,1]
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# Show images
imshow(torchvision.utils.make_grid(images))
# Print labels
print(' '.join(f'{classes[labels[j]]}' for j in range(4)))

# Step 5
Build the CNN Model and define the architecture.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)       # num of input channels (3-RGB), num of output channels (6), kernel size (5x5)
        self.pool = nn.MaxPool2d(2, 2)        # kernel size (2x2), stride (step size, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)      # num of input channels (6-from conv1), num of output channels (16), kernel size (5x5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 16 * 5 * 5 input features (flattened output of the last convolutional layer), 120 output features
        self.fc2 = nn.Linear(120, 84)         # -||-
        self.fc3 = nn.Linear(84, 10)          # -||-
        # Note - num of channels is the number of so-called 'filters'

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5) # flattening of the tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        # Data flow:      x      ->   conv1    ->  relu  ->   pool    ->   conv2    ->  relu  ->   pool   -> view  ->  fc1  -> relu  -> fc2  -> relu  -> fc3
        # Shape of x: N×3×32×32  ->  N×6×28×28 ->  const -> N×6×14×14 -> N×16×10×10 ->  const -> N×16×5×5 -> N×400 -> N×120 -> const -> N×84 -> const -> N×10

net = Net()

# Assuming 'net' is your model and 'inputs', 'labels' are your data
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move model to the device (GPU or CPU)
net.to(device)

# Example of moving tensors to GPU
# inputs, labels = inputs.to(device), labels.to(device)

# Step 6
Define a Loss Function and Optimizer:

In [ ]:
import torch.nn.functional as F

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Step 7
Train the Model

In [ ]:
for epoch in range(5):  # example with 5 epochs

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

# Testing
Evaluate the model's performance on the test data:

In [ ]:
correct = 0
total = 0

# We do not need to track gradients for evaluation
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)

        # Forward pass
        outputs = net(images)

        # Get the predicted class with the highest score
        _, predicted = torch.max(outputs.data, 1)

        # Update total and correct counts
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print accuracy
print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f} %')


Here is a nice visualization for the predictions on a batch of data:

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Ensure the model is on the GPU
net.to(device)

dataiter = iter(testloader)

In [ ]:
images, labels = next(dataiter)

# Move the inputs and labels to the GPU
images, labels = images.to(device), labels.to(device)

# Print images
imshow(torchvision.utils.make_grid(images.cpu()))  # Move images back to CPU for visualization
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]}' for j in range(4)))

# Get predictions from the model
outputs = net(images)

# Get the predicted class with the highest score
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]}' for j in range(4)))